In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# test

In [ ]:
path = '../input/'
test = pd.read_csv(path + 'test.csv')
print(test.shape)
test.head(3)

- shop_id, item_id 별

In [ ]:
print('submit해야하는 shop_id 개수 : ', len(test.shop_id.unique()))
print('submit해야하는 item_id 개수 : ', len(test.item_id.unique()))

# train

In [ ]:
train = pd.read_csv(path + 'sales_train.csv')
print(train.shape)
train.head(3)

## date

In [ ]:
train['date'] = pd.to_datetime(train.date,format = '%d.%m.%Y')
train['year'] = train.date.dt.year
train['month'] = train.date.dt.month
train['day'] = train.date.dt.day
train['weekday'] = train.date.dt.weekday #(Mon = 0 ~ Sun = 6)
train.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.barplot(train.month, train.item_cnt_day)

In [ ]:
sns.barplot(train.day, train.item_cnt_day)

In [ ]:
sns.barplot(train.weekday, train.item_cnt_day)

- 날짜에 따른 차이는 커보이지 않음

# shop_id

In [ ]:
shops = pd.read_csv(path + 'shops.csv')
print(shops.shape)
shops.head()

In [ ]:
shops['shop_len'] = shops.shop_name.apply(lambda x: str.replace(x, '!', '')).apply(lambda x: x.split(' ')).apply(len) #name_length
shops['city'] = shops.shop_name.apply(lambda x: str.replace(x, '!', '')).apply(lambda x: x.split(' ')[0])
shops['city'] = pd.Categorical(shops.city).codes

In [ ]:
train = pd.merge(train, shops.drop('shop_name', axis = 1), how = 'left')
train.head(3)

In [ ]:
sns.barplot(train.city, train.item_cnt_day)

In [ ]:
sns.barplot(train.shop_len, train.item_cnt_day)

# item_id

In [ ]:
items = pd.read_csv(path + 'items.csv')
item_categories = pd.read_csv(path + 'item_categories.csv')
items = pd.merge(items, item_categories, how = 'left')

del item_categories

In [ ]:
print(items.shape)
items.head(3)

In [ ]:
items['category1']= items.item_category_name.apply(lambda x : x.split('-')[0])
items['category2']= items.item_category_name.apply(lambda x : x.split('-')[0]).apply(lambda x : x.split(' ')[0])

items['category1'] = pd.Categorical(items.category1).codes
items['category2'] = pd.Categorical(items.category2).codes

In [ ]:
train.head()

In [ ]:
train = pd.merge(train, items.drop(['item_name', 'item_category_name'], axis = 1), how = 'left')
train.head(3)

In [ ]:
plt.figure(figsize = (15,10))
sns.barplot(train.item_category_id, train.item_cnt_day)

In [ ]:
sns.barplot(train.category1, train.item_cnt_day)

In [ ]:
sns.barplot(train.category2, train.item_cnt_day)

# item price

In [ ]:
train.item_price.describe()

In [ ]:
sns.distplot(train.item_price)

In [ ]:
plt.scatter(train.item_price, train.item_cnt_day)

In [ ]:
train['revenue'] = train['item_price'] *  train['item_cnt_day']

# data 정리

In [ ]:
train_sub = train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day':'sum'}).rename(columns = {'item_cnt_day' : 'item_cnt_month'}).reset_index()
y = train_sub.item_cnt_month
train_sub.head(3)

In [ ]:
train_sub.date_block_num.max()

In [ ]:
test['date_block_num'] = 34
df = pd.concat([train_sub.drop('item_cnt_month', axis = 1),test.drop('ID', axis = 1)])

df.shape

In [ ]:
df.head(3)

## groupby shop_id, item_id

In [ ]:
aa = train.groupby(['shop_id','item_id']).agg({'revenue':['sum','mean','std','nunique','median'],
                                        'item_price':['sum','mean','std','nunique','median']}).reset_index()
aa.columns = ['shop_id','item_id'] + ['_'.join(x) for x in aa.columns.ravel()[2:]]

aa.head(3)

In [ ]:
df = pd.merge(df, aa, how = 'left')

# Modeling

In [ ]:
X_train = df[df.date_block_num < 33]
y_train = y[df[df.date_block_num < 33].index]
X_valid = df[df.date_block_num == 33]
y_valid = y[df[df.date_block_num == 33].index]

X_test = df[df.date_block_num == 34] #test data

print('train shape X : {} y : {}'.format(X_train.shape, y_train.shape))
print('valid shape X : {} y : {}'.format(X_valid.shape, y_valid.shape))
print('test shape X : {}'.format(X_test.shape))

In [ ]:
import gc
del df
gc.collect()

In [ ]:
from lightgbm import LGBMRegressor
lgb = LGBMRegressor(n_jobs = -1)

lgb.fit(X_train, y_train, eval_metric='rmse', eval_set=[(X_train, y_train), (X_valid, y_valid)],
       verbose = 10, early_stopping_rounds=20)

In [ ]:
# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(lgb.feature_importances_, X_train.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features')
plt.tight_layout()
plt.show()

In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor(n_jobs = -1, n_estimators = 500) #default option

In [ ]:
xgb.fit(X_train, y_train ,eval_metric='rmse', eval_set=[(X_train, y_train), (X_valid, y_valid)],
         verbose = 10, early_stopping_rounds=20)

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(xgb.feature_importances_, X_train.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('XGBoost Features')
plt.tight_layout()
plt.show()

# submission

In [ ]:
y_pred_lgb = lgb.predict(X_test).clip(0,20)
y_pred_xgb = xgb.predict(X_test).clip(0,20)

In [ ]:
sub1 = pd.DataFrame({
    'ID' : test.index,
    'item_cnt_month' : y_pred_lgb
})

sub2 = pd.DataFrame({
    'ID' : test.index,
    'item_cnt_month' : y_pred_xgb
})

sub1.to_csv('20190724_lgb_pred.csv', index = False)
sub2.to_csv('20190724_xgb_pred.csv', index = False)